# 모델 모니터링

프로덕션 환경에 서비스로 배포한 모델을 모니터링하면 사용량을 추적하고 모델이 처리하는 요청을 살펴볼 수 있습니다. Azure Application Insights를 사용하여 모델 서비스 엔드포인트의 활동을 모니터링할 수 있습니다.

## 작업 영역에 연결합니다.

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## 배포할 모델 준비

이제 배포할 모델을 준비해야 합니다. 아래 코드를 실행하여 다음 단계를 수행합니다.

1. 데이터 세트를 만들고 등록합니다.
2. 데이터 세트를 사용하여 모델을 학습시킵니다.
3. 모델을 등록합니다.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from azureml.core import Dataset

# 기본 데이터 저장소에 데이터 파일 업로드
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 테이블 형식 데이터 세트 등록
print('Registering dataset...')
try:
    data_set = data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# 작업 영역에서 Azure ML 실험 만들기
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 당뇨병 데이터 세트 로드
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 학습된 모델 저장
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 실행 완료
run.complete()

# 모델 등록
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 가져오기
model = ws.models['diabetes_model']

print('Model trained and registered.')

## 모델을 웹 서비스로 배포

이제 등록된 모델을 웹 서비스로 배포할 수 있습니다.

먼저 배포 구성 파일용 폴더를 만듭니다.

In [ ]:
import os

folder_name = 'diabetes_service'

# 웹 서비스 파일용 폴더 만들기
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# 채점 스크립트용 경로 설정
script_file = os.path.join(experiment_folder,"score_diabetes.py")

이제 서비스가 새 데이터의 점수를 매기는 데 사용할 입력 스크립트가 필요합니다.

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# 서비스를 로드하면 호출됨
def init():
    global model
    # 배포된 모델 파일의 경로를 가져와서 로드
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 요청이 수신되면 호출됨
def run(raw_data):
    # 입력 데이터를 numpy 배열로 가져오기
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # 모델에서 예측 가져오기
    predictions = model.predict(np_data)
    
    # 데이터 및 예측이 기록되도록 인쇄
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # 각 예측(0 또는 1)에 해당하는 클래스 이름 가져오기
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # 예측을 JSON으로 반환
    return json.dumps(predicted_classes)

또한 서비스 환경용 Conda 구성 파일도 필요합니다.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# 모델의 종속성 추가(AzureML 기본값은 이미 포함되어 있음)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# 환경 구성을 .yml 파일로 저장
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# .yml 파일 인쇄
with open(env_file,"r") as f:
    print(f.read())

이제 서비스를 배포할 수 있습니다. 여기서는 서비스를 ACI(Azure Container Instance)로 배포합니다.

> **참고**: 배포는 몇 분 정도 걸릴 수 있습니다. 상태가 **정상**으로 표시될 때까지 기다리세요.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

# 채점 환경 구성
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

service_name = "diabetes-service-app-insights"
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

## Application Insights를 사용하도록 설정

다음으로는 서비스에서 Application Insights를 사용하도록 설정해야 합니다.

In [ ]:
# AppInsights 사용
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## 웹 서비스 사용

배포한 서비스는 클라이언트 애플리케이션에서 사용할 수 있습니다.

먼저 이러한 애플리케이션이 요청을 제출해야 하는 URL을 확인합니다.

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

엔드포인트 URI가 확인되면 애플리케이션은 HTTP 요청을 수행하여 JSON 또는 이진 형식으로 환자 데이터를 전송한 다음 예측된 클래스를 다시 수신할 수 있습니다.

> **팁**: 서비스 엔드포인트가 준비되지 않아 오류가 발생하는 경우 몇 초 기다렸다가 다시 시도하세요.

In [ ]:
import requests
import json

# 유추용 새 데이터 생성
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 배열을 JSON 문서의 직렬화 가능 목록으로 변환
input_json = json.dumps({"data": x_new})

# 콘텐츠 형식 설정
headers = { 'Content-Type':'application/json' }

# 예측 가져오기
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

이제 서비스 엔드포인트와 관련하여 기록된 데이터를 확인할 수 있습니다.
1. [Azure Portal](https://portal.azure.com)에서 Machine Learning 작업 영역을 엽니다.
2. **개요** 페이지에서 관련 **Application Insights** 리소스의 링크를 클릭합니다.
3. Application Insights 블레이드에서 **로그**를 클릭합니다. 

    > **참고**: Log Analytics를 처음 열었다면 **시작하기**를 클릭하여 쿼리 편집기를 열어야 할 수 있습니다. 쿼리 작성 방법을 설명하는 팁이 표시되면 닫습니다.

4. 다음 쿼리를 쿼리 편집기에 붙여넣고 **실행**을 클릭합니다.
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "diabetes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. 결과를 확인합니다. 처음에는 결과가 표시되지 않을 수 있습니다. ACI 웹 서비스가 Application Insights에 원격 분석을 전송하려면 최대 5분까지 걸릴 수 있기 때문입니다. 몇 분 정도 기다렸다가 기록된 데이터와 예측이 표시될 때까지 쿼리를 다시 실행합니다.
6. 로깅된 데이터를 검토한 후 Application Insights 쿼리 페이지를 닫습니다.

## 서비스 삭제

더 이상 필요하지 않은 서비스는 삭제해야 합니다.

> **참고**: 사용 중인 서비스는 즉시 삭제하지 못할 수도 있습니다.

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)

Application Insights를 사용하여 배포된 서비스를 모니터링하는 방법에 대한 자세한 내용은 [Azure Machine Learning 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights)를 참조하세요.